In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression, BayesianRidge
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
filenames = glob.glob('Datathon*.csv')

In [3]:
pre_dat = pd.DataFrame({})
for ind, filename in enumerate(filenames):
    print(ind)
    tmp_dat = pd.read_csv(filename)
    pre_dat = pre_dat.append(tmp_dat)    
pre_dat.columns = ['BET_ID', 'BET_TRANS_ID', 'MATCH_BET_ID', 'ACCOUNT_ID',
       'COUNTRY_OF_RESIDENCE_NAME', 'PARENT_EVENT_ID', 'EVENT_ID', 'MATCH',
       'EVENT_NAME', 'EVENT_DT', 'OFF_DT', 'BID_TYP', 'STATUS_ID',
       'PLACED_DATE', 'TAKEN_DATE', 'SETTLED_DATE', 'CANCELLED_DATE',
       'SELECTION_NAME', 'PERSISTENCE_TYPE', 'BET_PRICE', 'PRICE_TAKEN',
       'INPLAY_BET', 'BET_SIZE', 'PROFIT_LOSS']
peek_dat = pd.read_csv('DSMDatathon_samplefile.txt', sep='\t')
pre_dat = peek_dat.append(pre_dat)

0
1

/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)
/Users/feizhan/miniconda/envs/py3_env/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)



2
3
4


In [4]:
pre_dat.loc[:, 'BID_TYP'] = pre_dat.BID_TYP.str.strip(' ')
pre_dat.loc[:, 'STATUS_ID'] = pre_dat.STATUS_ID.str.strip(' ')
# correct the profit loss
c1 = ((pre_dat.BID_TYP == 'B') & (pre_dat.PROFIT_LOSS > 0))
c2 = ((pre_dat.BID_TYP == 'B') & (pre_dat.PROFIT_LOSS < 0))
c3 = ((pre_dat.BID_TYP == 'L') & (pre_dat.PROFIT_LOSS > 0))
c4 = ((pre_dat.BID_TYP == 'L') & (pre_dat.PROFIT_LOSS < 0))
pre_dat.loc[c1.values, 'correct_profit_loss'] = pre_dat.loc[c1.values, 'BET_SIZE'] * (pre_dat.loc[c1.values, 'PRICE_TAKEN'] -1)
pre_dat.loc[c2.values, 'correct_profit_loss'] = - pre_dat.loc[c2.values, 'BET_SIZE']
pre_dat.loc[c3.values, 'correct_profit_loss'] = pre_dat.loc[c3.values, 'BET_SIZE']
pre_dat.loc[c4.values, 'correct_profit_loss'] = - pre_dat.loc[c4.values, 'BET_SIZE'] * (pre_dat.loc[c4.values, 'PRICE_TAKEN'] -1)

In [5]:
submit = pd.read_csv('sample_submission_bet_size.csv')
submit.columns = ['ACCOUNT_ID', 'Prediction']

In [6]:
pre_dat = pre_dat.loc[(pre_dat.ACCOUNT_ID.isin(submit.ACCOUNT_ID)).values]

In [7]:
pre_dat = pre_dat.loc[(pre_dat.STATUS_ID == 'S').values]

In [19]:
pre_dat_by_account_event = pre_dat.groupby(['ACCOUNT_ID', 'EVENT_ID'])

In [139]:
win_loss = pre_dat_by_account_event['correct_profit_loss'].sum()

In [140]:
win_loss1 = win_loss > 0

In [141]:
maps = {True: 1, False: -1}

In [142]:
win_loss1 = win_loss1.map(maps)

In [143]:
win_loss1.head()

ACCOUNT_ID  EVENT_ID 
1000002     101093076   -1
            101093312   -1
            101128387    1
            101149398   -1
            101149516   -1
Name: correct_profit_loss, dtype: int64

In [144]:
win_loss1_by_account = win_loss1.groupby(level=0)

In [145]:
win_loss2 = win_loss1_by_account.sum()

In [146]:
win_loss2 = win_loss2.loc[submit.ACCOUNT_ID.tolist()]

In [147]:
win_loss2.head()

ACCOUNT_ID
1004240    -1
1009742    12
1003908     3
1020888    -1
1010074    -1
Name: correct_profit_loss, dtype: float64

In [148]:
win_loss2 = win_loss2.reset_index()

In [149]:
win_loss2.columns = ['Account_ID', 'Prediction']

In [150]:
win_loss2.Prediction.describe()

count    6373.000000
mean        0.951200
std         7.700753
min       -29.000000
25%        -3.000000
50%         0.000000
75%         4.000000
max        43.000000
Name: Prediction, dtype: float64

In [151]:
win_loss2 = win_loss2.fillna(-5)

In [152]:
win_loss2.to_csv('count_games_fill_minus5.csv', index=False)

In [153]:
win_loss2.head()

,Account_ID,Prediction
0,1004240,-1
1,1009742,12
2,1003908,3
3,1020888,-1
4,1010074,-1


In [154]:
win_loss2.shape

(7374, 2)

In [155]:
win_loss2.isnull().any()

Account_ID    False
Prediction    False
dtype: bool

In [8]:
four_team_pre_dat = pre_dat.loc[(pre_dat.MATCH.str.contains('Australia') |
                                 pre_dat.MATCH.str.contains('New Zealand') |
                                 pre_dat.MATCH.str.contains('India') |
                                 pre_dat.MATCH.str.contains('South Africa')).values]

In [9]:
four_team_pre_dat_by_account_event = four_team_pre_dat.groupby(['ACCOUNT_ID', 'EVENT_ID'])

In [10]:
win_loss = four_team_pre_dat_by_account_event['correct_profit_loss'].sum()

In [11]:
win_loss1 = win_loss > 0

In [12]:
maps = {True: 1, False: -1}

In [13]:
win_loss1 = win_loss1.map(maps)

In [14]:
win_loss1.head()

ACCOUNT_ID  EVENT_ID 
1000002     101093076   -1
            101149516   -1
            101150716    1
            101150834   -1
            101151878    1
Name: correct_profit_loss, dtype: int64

In [15]:
win_loss1_by_account = win_loss1.groupby(level=0)

In [16]:
win_loss2 = win_loss1_by_account.sum()

In [17]:
win_loss2 = win_loss2.loc[submit.ACCOUNT_ID.tolist()]

In [18]:
win_loss2 = win_loss2.reset_index()

In [19]:
win_loss2.columns = ['Account_ID', 'Prediction']

In [20]:
win_loss2 = win_loss2.fillna(-5)

In [21]:
win_loss2.to_csv('four_team_count_games_fill_minus5.csv', index=False)

In [22]:
win_loss2.isnull().any()

Account_ID    False
Prediction    False
dtype: bool